# 파인튜닝 튜토리얼

<img src="https://i.imgur.com/j3zhyLz.png" width="500px" />

라이브러리 설치 후 런타임 재시작

In [ ]:
!pip install transformers datasets accelerate -qqq

## 1.작업정의

사전 학습된 언어생성 모델을 가져와서 질의 응답 데이터셋을 가지고 파인튜닝하기

## 2.학습 데이터 로딩

## 3.토크나이저 로딩

## 4.모델 로딩

## 5.학습 데이터 전처리

### 5.1. 토크나이징

#### 기본용법

```python
[{'from': 'human',
  'markdown': None,
  'text': None,
  'value': 'Sure, I can do that. What new technology would you like me to review?'},
 {'from': 'gpt',
  'markdown': None,
  'text': None,
  'value': 'Samsung S22 Ultra...'},
```

In [ ]:
"""
HUMAN: ~~~~  \n
CHATBOT: ~~~~ \n
"""

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
	conversations = examples["conversations"]
	text = ''
	for item in conversations:
		if item['from'] == "human":
			text += f"HUMAN: {item['value']} \n"
		else:
			text += f"CHATBOT: {item['value']} \n"
	return tokenizer(text, padding="max_length", truncation=True, max_length=512)


tokenized_datasets = train_dataset.map(
		tokenize_function,
		remove_columns=["conversations", "id"]
)
tokenized_datasets



### 5.2. 배치 처리 (DataCollator)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
data_collator

## 6.모델 학습

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
	output_dir="./gpt2_finetuned",
	overwrite_output_dir=True,
	num_train_epochs=1,
	per_device_train_batch_size=2,
	save_steps=1000,
	save_total_limit=2,
)

# Trainer 객체를 생성
trainer = Trainer(
	model=model,
	args=training_args,
	data_collator=data_collator,
	train_dataset=tokenized_datasets,
)

# 학습 시작
trainer.train()

In [ ]:
# 1) 질의 문장
input_text = "인공지능에 대해 알려주세요"


# 2) 토큰화하고 텐서로 변환합니다.
input_ids = tokenizer.encode(input_text, return_tensors="pt")
input_ids = input_ids.to('cuda')

# 3) 생성 옵션을 설정하고 텍스트를 생성합니다.
max_length = 500
model = model.to("cuda")
sample_outputs = model.generate(input_ids, do_sample=True, max_length=max_length, temperature=0.7)

# 4) 생성된 텍스트를 디코딩합니다.

print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))